In [210]:
matr = np.genfromtxt('4x8adk-fig8.csv', dtype='int', delimiter=',')
a=Adinkra(matr)
wb=[254,128]
wf=[240]
print(a.hymns(wb,wf))

({rhoB**2*rhoF: 4, -rhoB**2*rhoF: 4}, {rhoB**2*rhoF: 4, -rhoB**2*rhoF: 4})


In [203]:
import numpy as np
import sympy as sp

class Adinkra:

    def __init__(obj,matr):
        obj.ADJMatrix = matr
        obj.open=len(matr)
        obj.closed=len(matr[0])
        obj.nodes=obj.open+obj.closed
        obj.colors=abs(np.max(matr)) #wont work if all of the last color is negative (all of the connections of the highest-indexed color are dashed). Does this ever happen?
        obj.LMatrices=np.zeros((obj.colors,obj.open,obj.closed))
        for i in range(0,obj.colors):
            obj.LMatrices[i]=Adinkra.LColBlock(obj,i+1)
        obj.hand=obj.colors%2
        
    def LColBlock(a,color):
        h=a.open;
        w=a.closed;
        lcol=np.zeros((h,w));
        for i in range(0,h):
            for j in range(0,w):
                if (a.ADJMatrix[i,j]==color):
                    lcol[i,j]=1
                elif (a.ADJMatrix[i,j]==color*-1):
                    lcol[i,j]=-1
        return lcol
    
    def liftMatrix(a,m,w):
        d=a.closed
        M=sp.Matrix(np.identity(d,dtype=int))
        for i in w:
            bin=i
            for j in range(d-1,-1,-1):
                if (bin>=(2**j)):
                    M[j,j]*=m
                    bin-=2**j
        return M
    
    def LTilde(a,color,mb,mf,wb,wf):
        L = a.LMatrices[color-1]
        L = (a.liftMatrix(mb,wb))*L*(a.liftMatrix(1/mf,wf))
        return L
    
    def RTilde(a,color,mub,muf,wb,wf):
        R = np.transpose(a.LMatrices[color-1])
        R = (a.liftMatrix(muf,wf))*R*(a.liftMatrix(1/mub,wb))
        return R
    
    def BanchL(a,mb,mub,mf,muf,wb,wf):
        I=a.colors
        BL = a.LTilde(I,mb,mf,wb,wf)
        for i in range(I-1,0,-1):
            if (i%2==1):
                BL = BL*(a.RTilde(i,mub,muf,wb,wf))
            else:
                BL = BL*(a.LTilde(i,mb,mf,wb,wf))
        return BL
    
    def BanchR(a,mb,mub,mf,muf,wb,wf):
        I=a.colors
        BR = a.RTilde(I,mub,muf,wb,wf)
        for i in range(I-1,0,-1):
            if (i%2==1):
                BR = BR*(a.LTilde(i,mb,mf,wb,wf))
            else:
                BR = BR*(a.RTilde(i,mub,muf,wb,wf))
        return BR
    
    def hymns(a,wb,wf):
        mb=sp.Symbol('mb')
        rhoB=sp.Symbol('rhoB')
        mub=mb/rhoB
        muf=sp.Symbol('muf')
        rhoF=sp.Symbol('rhoF')
        mf=muf/rhoF
        BL = a.BanchL(mb,mub,mf,muf,wb,wf)
        BR = a.BanchR(mb,mub,mf,muf,wb,wf)
        if (a.hand==0):
            return BL.eigenvals(), BR.eigenvals()
        else:
            return 0
    